# Heart Disease Interactive Dashboard (Matplotlib + Panel)

Interactive dashboard using Panel and Matplotlib to explore the heart disease dataset.

In [2]:
pip install panel

  Using cached panel-1.8.2-py3-none-any.whl.metadata (15 kB)
  Using cached bleach-6.3.0-py3-none-any.whl.metadata (31 kB)
  Using cached bokeh-3.8.0-py3-none-any.whl.metadata (10 kB)
  Using cached linkify_it_py-2.0.3-py3-none-any.whl.metadata (8.5 kB)
  Using cached markdown-3.10-py3-none-any.whl.metadata (5.1 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdit_py_plugins-0.5.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached pandas-2.3.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached param-2.2.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached pyviz_comms-3.0.6-py3-none-any.whl.metadata (7.7 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached narwhals-2.10.1-py3-none-any.whl.metadata (11 kB)
  Using 


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
!pip install panel jupyter_bokeh matplotlib seaborn --upgrade


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
# 📦 1. Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import panel as pn
import seaborn as sns

# Initialize Panel extension with all requirements
pn.extension('matplotlib', 'plotly', sizing_mode='stretch_width')

# Set a consistent style
sns.set_style("whitegrid")
plt.style.use('seaborn')

OSError: 'seaborn' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

In [6]:
# Load dataset (expects 'heart.csv' to be in the notebook working directory)
# Replace the path if your CSV is located elsewhere
df = pd.read_csv("heart.csv")

df.head()

,Unnamed: 0,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [7]:
# Create filter widgets
sex_filter = pn.widgets.RadioButtonGroup(name='Sex', options=['All', 'Male', 'Female'], value='All')
# Build chest pain options from the dataset so the widget adapts to the CSV
cp_values = sorted(df['ChestPain'].dropna().unique().tolist())
cp_options = ['All'] + [str(v) for v in cp_values]
cp_filter = pn.widgets.Select(name='Chest Pain Type (cp)', options=cp_options, value='All')

sex_filter, cp_filter

(RadioButtonGroup(name='Sex', options=['All', 'Male', 'Female'], value='All'),
 Select(name='Chest Pain Type (cp)', options=['All', 'asymptomatic', ...], value='All'))

In [ ]:
def filter_data(sex_choice, cp_choice):
    """
    Filters the dataframe based on sex_choice and cp_choice.
    """
    data = df.copy()
    # Filter by sex: dataset uses 1 for male, 0 for female
    if sex_choice != 'All':
        sex_val = 1 if sex_choice == 'Male' else 0
        data = data[data['sex'] == sex_val]
    # Filter by chest pain type
    if cp_choice != 'All':
        try:
            cp_val = int(cp_choice)
            data = data[data['cp'] == cp_val]
        except Exception:
            # If conversion fails, do not filter
            pass
    return data


def plot_dashboard(sex_choice, cp_choice):
    """Return a Matplotlib Figure for the current widget selections."""
    data = filter_data(sex_choice, cp_choice)

    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    # Chart 1: Age distribution
    axs[0, 0].hist(data['Age'].dropna(), bins=15, color='#4C72B0', edgecolor='black')
    axs[0, 0].set_title('Age Distribution')
    axs[0, 0].set_xlabel('Age')
    axs[0, 0].set_ylabel('Count')

    # Chart 2: Average Cholesterol by Heart Disease Target
    if 'chol' in data.columns and 'target' in data.columns:
        chol_mean = data.groupby('target')['chol'].mean().reindex([0, 1])
        chol_mean_na = chol_mean.fillna(0)
        axs[0, 1].bar(['No Disease', 'Disease'], chol_mean_na, color=['#55A868', '#C44E52'])
        axs[0, 1].set_title('Average Cholesterol by Heart Disease')
        axs[0, 1].set_ylabel('Mean Cholesterol')
    else:
        axs[0, 1].text(0.5, 0.5, 'chol or target column missing', ha='center')
        axs[0, 1].set_axis_off()

    # Chart 3: Resting Blood Pressure distribution
    if 'trestbps' in data.columns:
        axs[1, 0].hist(data['trestbps'].dropna(), bins=15, color='#8172B2', edgecolor='black')
        axs[1, 0].set_title('Resting Blood Pressure Distribution')
        axs[1, 0].set_xlabel('Resting BP')
        axs[1, 0].set_ylabel('Count')
    else:
        axs[1, 0].text(0.5, 0.5, 'trestbps column missing', ha='center')
        axs[1, 0].set_axis_off()

    # Chart 4: Heart Disease case counts
    if 'target' in data.columns:
        counts = data['target'].value_counts().reindex([0, 1]).fillna(0)
        axs[1, 1].bar(['No Disease', 'Disease'], counts, color=['#2E91E5', '#E15F99'])
        axs[1, 1].set_title('Heart Disease Case Counts')
        axs[1, 1].set_ylabel('Count')
    else:
        axs[1, 1].text(0.5, 0.5, 'target column missing', ha='center')
        axs[1, 1].set_axis_off()

    plt.tight_layout()
    return fig

In [ ]:
# Create the dashboard with explicit sizing
dashboard = pn.Column(
    '# ❤️ Heart Disease Interactive Dashboard',
    pn.Row(sex_filter, cp_filter),
    pn.pane.Matplotlib(
        pn.bind(plot_dashboard, sex_choice=sex_filter, cp_choice=cp_filter),
        height=600,
        width=800
    ),
    sizing_mode='stretch_width'
)

# Display the dashboard
dashboard.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'417aa9bc-751f-4b0b-8929-fc5b5ec3454e': {'version…